In [1]:
import os
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from display_graph import display_graph
import keyring

In [3]:
# Set up OpenAI APi Key
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')

# Initialize the OpenAI model
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model='gpt-4o-mini',
    temperature=0.7,        # Adjust for creativity
    max_tokens=50           # Set maximum token length for responses
)

In [5]:
# Define the state structure
class State(TypedDict):
    input: str
    draft_content: str
    
# Define node functions
def create_draft(state: State):
    print("---Generating Draft with ChatOpenAI---")
    
    # Prepare the prompt for generating the blog content
    prompt = f"Write a blog post on the topoc: {state['input']}"
    
    # Call the LangChain ChatOpenAI instance to generate the draft
    response = llm.invoke([{"role":"user", "content":prompt}])
    
    # Extract the generated content
    state["draft_content"] = response.content
    print(f"Generated Draft:\n{state['draft_content']}")
    return state

In [6]:
def review_draft(state: State):
    print("---Reviewing Draft---")
    print(f"Draft for review:\n{state['draft_content']}")
    return state

def publish_content(state: State):
    print("---Publishing Content---")
    print(f"Published Content:\n{state['draft_content']}")
    return state

In [7]:
# Build the graph
builder = StateGraph(State)
builder.add_node("create_draft", create_draft)
builder.add_node("review_draft", review_draft)
builder.add_node("publish_content", publish_content)

# Define the flow
builder.add_edge(START, "create_draft")
builder.add_edge("create_draft", "review_draft")
builder.add_edge("review_draft", "publish_content")
builder.add_edge("publish_content", END)

In [8]:
# Set up memory and breakpoints
memory = MemorySaver()
graph = builder.compile(
    checkpointer=memory,
    interrupt_before=["publish_content"]
)

# Display the graph
display_graph(graph)

d:\projects\github\learning\llm\langgraph\langgraph_blueprint\ch9/graphs/graph_69256.png


In [9]:
# Run the graph
config = {"configurable": {"thread_id":"thread-1"}}
initial_input = {"input": "The importance of AI in modern content creation."}

# Run the first part until the review step
thread = {"configurable":{"thread_id": "1"}}

for event in graph.stream(initial_input, thread, stream_mode="values"):
    print(event)
    
# Pausing for human review
user_approaval = input("Do you approve the draft for publishing? (yes/no/modification): ")

if user_approaval.lower() == "yes":
    # Proceed to publish content
    for event in graph.stream(None, thread, stream_mode="values"):
        print(event)
elif user_approaval.lower() == "modification":
    updated_draft = input("Please modify the draft content:\n")
    memory.update({"draft_content": updated_draft})     # Update memory with new content
print("Draft updated by the editor.")

{'input': 'The importance of AI in modern content creation.'}
---Generating Draft with ChatOpenAI---
Generated Draft:
# The Importance of AI in Modern Content Creation

In today’s fast-paced digital landscape, content is king. Businesses, brands, and creators are constantly seeking innovative ways to engage their audiences and stand out in a crowded marketplace. As the demand for high
{'input': 'The importance of AI in modern content creation.', 'draft_content': '# The Importance of AI in Modern Content Creation\n\nIn today’s fast-paced digital landscape, content is king. Businesses, brands, and creators are constantly seeking innovative ways to engage their audiences and stand out in a crowded marketplace. As the demand for high'}
---Reviewing Draft---
Draft for review:
# The Importance of AI in Modern Content Creation

In today’s fast-paced digital landscape, content is king. Businesses, brands, and creators are constantly seeking innovative ways to engage their audiences and stand o